In [8]:
!pip install unstructured
!pip install unstructured[docx]
!pip install -q cassio datasets langchain tiktoken
!pip install sentence-transformers

In [ ]:
!pip install PyPDF2

##  Directory Loader

In [1]:
from langchain.document_loaders import DirectoryLoader

In [2]:
loader = DirectoryLoader('./content', show_progress=True)
docs = loader.load()

  0%|          | 0/20 [00:00<?, ?it/s]

g:\Github\langchain\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


In [ ]:
for i in range(len(docs)):
    print(docs[i].metadata)

In [53]:
# markdown loader
# from langchain.document_loaders import UnstructuredMarkdownLoader
# loader = UnstructuredMarkdownLoader('AIconsultingdevelopmentservices.md',mode="elements" , show_progress=True)

In [54]:
# data = loader.load()
# for i in range(len(data)):
#     print(f"{i}\t" ,data[i].page_content)

In [55]:
# data

In [3]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./pdfs', show_progress=True)

In [5]:
# d = loader.load_and_split()
# d[5].page_content
# for i in range(len(d)):
#     print(d[i].metadata)

100%|██████████| 19/19 [00:12<00:00,  1.47it/s]


In [29]:
doc_dict = dict()

## PyPDF Loader

In [1]:
# open folder
import os
from langchain.document_loaders import PyPDFLoader
doc_dict = dict()
for file in os.listdir("./pdfs"):
    if file.endswith(".pdf"):
        loader = PyPDFLoader(f"./pdfs/{file}")
        x = loader.load_and_split()
        doc_dict[file] = x

In [2]:
sum = 0
for i in doc_dict.keys():
    # print(i, "\t", len(doc_dict[i]))
    sum += len(doc_dict[i])
sum

116

In [3]:
doc_list = list()
for i in doc_dict.values():
    for j in i:
        doc_list.append(j)

In [4]:
print("Total number of pages :" , len(doc_list))

Total number of pages : 116


In [5]:
f"{doc_list[0].metadata['source'].split('/')[-1]}_{doc_list[0].metadata['page']}"

'Aboutus.pdf_0'

## Text splitter for text

In [23]:
# loading pages using pyPDF2
# from langchain.document_loaders import PyPDFLoader
# loader = PyPDFLoader('About us.pdf')
# docs = loader.load_and_split()

In [45]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=100,
)

In [ ]:
for _ in range(len(doc_list)):
    print(doc_list[_].metadata)

In [47]:
doc_dict = dict()

In [49]:
doc_dict[docs[0].metadata['source']] = text_splitter.split_documents(docs)

In [ ]:
doc_dict

In [88]:
from tqdm import tqdm

In [89]:
chunklist = []

for i in tqdm(range(len(docs))):
    texts = text_splitter.split_text(docs[i].page_content)
    chunklist.append(texts)

100%|██████████| 20/20 [00:00<00:00, 246.68it/s]


In [105]:
len(chunklist[0][1].split())

344

## Text splitter for PDF

In [133]:
split_docs = text_splitter.split_documents(docs)

In [144]:
len(split_docs[1].page_content)

2514

In [106]:
# chunklist[2]

In [80]:
from langchain.embeddings import HuggingFaceEmbeddings

## Working with vector store

In [6]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI

from datasets import load_dataset
import cassio

{
  "clientId": "vojbKbGDdCRrSKhkRvABraBr",
  "secret": "qZ23HXAoyrqzNTm41BJyl5Y0ujrO65lFXcEJ8GOolu_jUa4B_lR+LAj,2_Fbiw+67frXW-6E+YHk1oPb63Y7MZ+GZ008C7gToJhS2w9I1i1Q-qspT97kbfS84o6pLs.R",
  "token": "AstraCS:vojbKbGDdCRrSKhkRvABraBr:863a31ec15ff213b2e3a152f1776299cb4dfde364eec30eebf783091e2fe0bf8"
}

In [83]:
# from typing_extensions import Concatenate
# raw_text = ''

# for i in range(len(docs)):
#     raw_text += docs[i].page_content

In [ ]:
CONNECTION_STRING = "postgresql+psycopg2://harrisonchase@localhost:5432/test"

In [7]:
from langchain_community.vectorstores import AstraDB,PGVector

In [8]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("PGVECTOR_HOST", "localhost"),
    port=int(os.environ.get("PGVECTOR_PORT", "5433")),
    database=os.environ.get("PGVECTOR_DATABASE", "postgres"),
    user=os.environ.get("PGVECTOR_USER", "postgres"),
    password=os.environ.get("PGVECTOR_PASSWORD", "admin"),
)
COLLECTION_NAME = "test"

In [ ]:
# import faiss
# def create_faiss_index(embeddings, dimension):
#     index = faiss.IndexFlatL2(dimension)  # For cosine similarity with L2 distance
#     index.add(embeddings)
#     return index

# embeddings = embedding.embed_query("hello")
# create_faiss_index(embeddings, 512)

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
model = HuggingFaceEmbeddings()
COLLECTION_NAME = "test"
db = PGVector.from_documents(
    embedding=model,
    documents=doc_list,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [43]:
vector = model.embed_query('AWS sagemaker')

In [44]:
db.similarity_search_by_vector(vector, k=3)

ValueError: Collection not found

In [41]:
import psycopg2
from dotenv import load_dotenv
import os
load_dotenv()
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'admin'
DB_HOST = 'localhost'
DB_PORT = '5433'
def execute_query(user_query, model):
    query_embedding = model.embed_query(user_query)
    conn = psycopg2.connect(database=DB_NAME,
                                user=DB_USER,
                                password=DB_PASSWORD,
                                host=DB_HOST,
                                port= DB_PORT)
    cursor = conn.cursor()
    try:
        

        cursor.execute("""
            SELECT public.cosine_similarity(embedding, %(vector)s::vector) AS similarity
            FROM langchain_pg_embedding;
        """, {'vector': query_embedding})

        results = cursor.fetchall()
        return results

    except Exception as e:
        print(f"Error executing query: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [ ]:
str(model.embed_query('AWS sagemaker'))

In [42]:
execute_query("hello", model)

[]

In [ ]:
str(model.embed_query("aws cloud sagemaker azure google cloud platform"))

In [10]:
# db.add_documents(doc_list)

In [55]:
# cassio.init(database_id=ASTRA_DB_ID, token=ASTRA_DB_TOKEN)
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

g:\Github\langchain\langchain\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [61]:
#import hugging face embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

In [123]:
del vstore

In [ ]:
embedding.embed_documents(doc_list)

In [64]:
ASTRA_DB_ENDPOINT = "https://7c0f8dc7-64fc-4933-8e6a-54012dd88d6f-centralindia.apps.astra.datastax.com"
ASTRA_DB_TOKEN = "AstraCS:aXvdbOYgcdTfbqbzLtUraZmr:3d5c04059d72e98b77260d4f74956bbb05c619e416fe02a3a88f938ca56b722b"
vstore = AstraDB(
    embedding=embedding,
    collection_name="test",
    token=ASTRA_DB_TOKEN,
    api_endpoint=ASTRA_DB_ENDPOINT
)

In [66]:
inserted_ids = vstore.add_documents(doc_list)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 116 documents.


In [81]:
vector = embedding.embed_query('Please tell me about your services')

In [ ]:
str(vector)

In [77]:
ASTRA_DB_ID = "7c0f8dc7-64fc-4933-8e6a-54012dd88d6f"
# import cassio library
import cassio

cassio.init(database_id=ASTRA_DB_ID, token=ASTRA_DB_TOKEN)

In [80]:
from langchain.vectorstores.cassandra import Cassandra
astra_vector_cass = Cassandra(
    embedding=embedding,
    table_name="test", 
    keyspace=None,
    session=None
)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Undefined column name vector in table "default_keyspace".test"

In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("Enter your query(or type quit to stop): ")
    else :
        query_text = input("What is your next query (or type quit to stop) : ")
    
    if query_text.lower() == 'quit':
        break
    
    if query_text == '':
        continue

    first_question = False

    print(f"{query_text}\nSearching for similar documents...")
    answer = vstore.